## Trabalho de conclusão do curso de Especialização em Ciência de Dados da Facens
## Faculdade de Engenharia de Sorocaba

**Alunos:** 
<p>
<p>Dani Shizue Sakamoto
<p>Mariana de Castro Pires Tavares Albuquerque
<p>Tan Kin Wah

<h2><center>XGBoost Hyperparameter Search</center></h2>

Tendo verificado, com o Pycaret, que o XGboost era o modelo de melhor performance para o problema, efetuamos uma tentativa de hiperparametrização do mesmo, utilizando a biblioteca Xgboost.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn import model_selection



In [ ]:
params = {
    # Parameters that we are going to tune.
    'max_depth': range(3,10,2),
    'min_child_weight': range(1,6,2),
    'eta': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.5, 1],
    'colsample_bytree': [0.5, 1],
    # Other parameters
    'objective': 'multi:softmax',
    'num_class' :3,
    'eval_metric': 'merror',
    'seed': 39
}

In [ ]:
import numpy as np
data = np.load('adience-embeddings-2.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

In [ ]:
model = XGBClassifier()
model.fit(trainX, trainy)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
y_pred = model.predict(testX)

In [ ]:
print('Classification Report Teste')
print(metrics.classification_report(testy, y_pred))

Classification Report Teste
              precision    recall  f1-score   support

       adult       0.94      0.99      0.96      2607
       child       0.98      0.97      0.97      1715
    teenager       0.93      0.68      0.79       480

    accuracy                           0.95      4802
   macro avg       0.95      0.88      0.91      4802
weighted avg       0.95      0.95      0.95      4802



In [ ]:
#hiperparametrização de num trees e max depth
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
matplotlib.use('Agg')
import numpy
# load data
# encode string class values as integers
label_encoded_y = LabelEncoder().fit_transform(trainy)
# grid search
model = XGBClassifier()
n_estimators = [50, 100, 150, 200]
max_depth = [2, 4, 6, 8]
print(max_depth)
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(trainX, label_encoded_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot results
scores = numpy.array(means).reshape(len(max_depth), len(n_estimators))
for i, value in enumerate(max_depth):
    plt.plot(n_estimators, scores[i], label='depth: ' + str(value))
plt.legend()
plt.xlabel('n_estimators')
plt.ylabel('Log Loss')
plt.savefig('n_estimators_vs_max_depth.png')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self

[2, 4, 6, 8]
Fitting 10 folds for each of 16 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 32.1min finished


Best: -0.153352 using {'max_depth': 4, 'n_estimators': 150}
-0.258127 (0.011673) with: {'max_depth': 2, 'n_estimators': 50}
-0.215284 (0.012103) with: {'max_depth': 2, 'n_estimators': 100}
-0.196583 (0.012241) with: {'max_depth': 2, 'n_estimators': 150}
-0.186039 (0.012721) with: {'max_depth': 2, 'n_estimators': 200}
-0.181268 (0.010056) with: {'max_depth': 4, 'n_estimators': 50}
-0.157073 (0.012262) with: {'max_depth': 4, 'n_estimators': 100}
-0.153352 (0.014409) with: {'max_depth': 4, 'n_estimators': 150}
-0.156682 (0.015364) with: {'max_depth': 4, 'n_estimators': 200}
-0.160205 (0.013167) with: {'max_depth': 6, 'n_estimators': 50}
-0.161891 (0.016917) with: {'max_depth': 6, 'n_estimators': 100}
-0.167182 (0.018250) with: {'max_depth': 6, 'n_estimators': 150}
-0.171113 (0.019016) with: {'max_depth': 6, 'n_estimators': 200}
-0.170349 (0.015328) with: {'max_depth': 8, 'n_estimators': 50}
-0.173559 (0.017478) with: {'max_depth': 8, 'n_estimators': 100}
-0.176476 (0.018777) with: {'max_d

NameError: name 'pyplot' is not defined

In [ ]:
model = XGBClassifier(n_estimators = 4, max_depth = 150)
model.fit(trainX, trainy)
ypred = model.predict(testX)

In [ ]:
train_pred = model.predict(trainX)

print('Classification Report Treinamento')
print(metrics.classification_report(trainy, train_pred))

print()
print('Classification Report Teste')
print(metrics.classification_report(testy, ypred)) #modelo hiperparametrizado não ficou tão bom quanto o automático

Classification Report Treinamento
              precision    recall  f1-score   support

       adult       1.00      1.00      1.00      6098
       child       1.00      1.00      1.00      4075
    teenager       1.00      0.99      1.00      1141

    accuracy                           1.00     11314
   macro avg       1.00      1.00      1.00     11314
weighted avg       1.00      1.00      1.00     11314


Classification Report Teste
              precision    recall  f1-score   support

       adult       0.90      0.97      0.94      2607
       child       0.95      0.94      0.95      1715
    teenager       0.87      0.53      0.66       480

    accuracy                           0.92      4802
   macro avg       0.91      0.81      0.85      4802
weighted avg       0.92      0.92      0.91      4802



Verificamos, no caso em tela, que os resultados obtidos após a hiperparametrização foram inferiores ao modelo original. Desse modo, utilizamos o modelo criado inicialmente.